In [1]:
import jieba
import re
import pandas
from jieba import posseg
from sklearn.naive_bayes import MultinomialNB


In [2]:
jieba.enable_parallel() # 开启并行分词
jieba.enable_paddle() # 加载词性标注模块

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.659 seconds.
Prefix dict has been built successfully.
W1203 23:55:49.571306 217434 init.cc:157] AVX is available, Please re-compile on local machine
Paddle enabled successfully......


In [3]:
rawdata_path='./data/0.xlsx'

with open(rawdata_path, 'r',encoding='ISO-8859-1') as fp:
    sheets=pandas.read_excel(rawdata_path,sheet_name=None,index_col=0)

sheet1=sheets['Sheet1'] # 表格一，好评
sheet2=sheets['Sheet2'] # 表格二，差评

In [4]:
sheet_gnwd_tmp=sheet1.get(sheet1.columns[5:8].append(sheet1.columns[2:3]) )
sheet1_gnwd=sheet_gnwd_tmp[sheet_gnwd_tmp['编号'].notnull()]

In [5]:
sheet1_gnwd

,美学维度,情感得分,编号,评论内容
productId,,,,
64986275944,颜值很高,1.0,1.0,颜值很高，穿着很舒服，重量轻，很轻便
64986275944,鞋样子很好看,1.0,2.0,鞋样子很好看，号码也很足，穿着非常轻很舒服，快递也很快，在京东买东西就是放心
69950639664,颜色很正，涂上很显女王气质,1.0,3.0,很喜欢迪奥的口红，真的很好用，颜色很正，不容易起皮，涂上很显女王气质，物流服务态度很好。
69950639664,颜色很正，薄涂很显色度，,1.0,4.0,质感非常好，拿在手里挺有分量的感觉，颜色很正，薄涂很显色度，日常不化妆也能涂，滑润不干唇，个...
69950639664,感觉哑光款更适合自己，更加时尚更打底具气场，颜色很正的大红色，很显气质,1.0,5.0,之前在免税店买了一支滋润款的很好用，这次在京东试着买了哑光款的，正品无疑，感觉哑光款更适合自...
...,...,...,...,...
59158524459,高端大气上档次,1.0,1112.0,高端大气上档次，做工精细龙头，配件齐全
59158524459,个人感觉应该很好看,1.0,1113.0,洗菜盆质量真的很好，很厚实，材质也好，和描述的一样，四季沐歌的东西质量很好，我家的水龙头是这...
59158524459,宝贝很好，外观颜值非常高,1.0,1114.0,京东自营真的太好了，价格透明，送货太快了，虽然不着急用，但是上午拍下午就到购物体验太爽了，宝...


In [6]:
sheet_gnwd_tmp=sheet2[sheet2.columns[5:8].append(sheet2.columns[2:3])]
sheet2_gnwd=sheet_gnwd_tmp[sheet_gnwd_tmp['编号'].notnull()]


In [7]:
sheet2_gnwd

,美学维度,情感得分,编号,评论内容
productId,,,,
67163383132,实物颜色与买的时候严重不符,-1.0,1117.0,实物颜色与买的时候严重不符
63767275586,和专柜买的差别很大，颜色不正，004号偏粉色，而且几乎看不出来，非常失望的一次购物,-1.0,1118.0,和专柜买的差别很大，颜色不正，004号偏粉色，而且几乎看不出来，非常失望的一次购物
29048916380,亲！都别再上当受骗了，口红?颜色和色号都不一样.不是色差问题，而是完全不是一个色号，,-1.0,1119.0,亲！都别再上当受骗了，口红?颜色和色号都不一样，和客服联系爱搭不理，谁买谁上当！不是色差问题...
100007843983,图片跟实物区别太大了 不好看,-1.0,1120.0,图片跟实物区别太大了 不好看
100007843983,也不是图片上的颜色,-1.0,1121.0,特别干涂起来也不是图片上的颜色
...,...,...,...,...
61588121796,就一只唇膏，色号跟实体不符,-1.0,1196.0,就一只唇膏，色号跟实体不符
29986664062,产品颜色：南瓜色,0.0,1197.0,特别掉色，早上涂的口红中途不吃东西到中午就没色了。\n产品颜色：南瓜色
29986664062,特别特别的不好 千万别买 颜色也不好看,-1.0,1198.0,特别特别的不好 千万别买 颜色也不好看 抹在嘴上还特别特别的粘


In [8]:
with open('cn_stopwords.txt','r') as fp:
    stopwords_list=set([w.strip() for w in fp.readlines()])

In [9]:
def seg_sentence(sentence, stop_words):
    stop_flag = ['x', 'c', 'u', 'd', 'p', 't', 'uj', 'm', 'f', 'r'] # 见https://github.com/fxsjy/jieba，词性标注章节
    sentence_seged = posseg.cut(re.sub('\n', '', sentence))
    outstr = []
    for word, flag in sentence_seged:
        if word not in stop_words and flag not in stop_flag:
            outstr.append(word)
    return ' '.join(outstr)


In [10]:
sheet3_gnwd=sheet1_gnwd.append(sheet2_gnwd)

In [11]:
sheet3_gnwd

,美学维度,情感得分,编号,评论内容
productId,,,,
64986275944,颜值很高,1.0,1.0,颜值很高，穿着很舒服，重量轻，很轻便
64986275944,鞋样子很好看,1.0,2.0,鞋样子很好看，号码也很足，穿着非常轻很舒服，快递也很快，在京东买东西就是放心
69950639664,颜色很正，涂上很显女王气质,1.0,3.0,很喜欢迪奥的口红，真的很好用，颜色很正，不容易起皮，涂上很显女王气质，物流服务态度很好。
69950639664,颜色很正，薄涂很显色度，,1.0,4.0,质感非常好，拿在手里挺有分量的感觉，颜色很正，薄涂很显色度，日常不化妆也能涂，滑润不干唇，个...
69950639664,感觉哑光款更适合自己，更加时尚更打底具气场，颜色很正的大红色，很显气质,1.0,5.0,之前在免税店买了一支滋润款的很好用，这次在京东试着买了哑光款的，正品无疑，感觉哑光款更适合自...
...,...,...,...,...
61588121796,就一只唇膏，色号跟实体不符,-1.0,1196.0,就一只唇膏，色号跟实体不符
29986664062,产品颜色：南瓜色,0.0,1197.0,特别掉色，早上涂的口红中途不吃东西到中午就没色了。\n产品颜色：南瓜色
29986664062,特别特别的不好 千万别买 颜色也不好看,-1.0,1198.0,特别特别的不好 千万别买 颜色也不好看 抹在嘴上还特别特别的粘


In [12]:
dimension,score,index,comment=[x for x in sheet3_gnwd]

In [13]:
import numpy as np
gnwd_all=sheet3_gnwd.values


In [14]:
gnwd_dimension=gnwd_all[:,0]
gnwd_score=gnwd_all[:,1]
gnwd_comment=gnwd_all[:,3]

In [15]:
gnwd_dimension_list=gnwd_dimension.tolist()

In [16]:
x=[seg_sentence(x,stop_words=stopwords_list) for x in gnwd_dimension_list]
y=gnwd_score.tolist()

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer()

In [19]:
xtrain=vector.fit_transform(x_train)
xtest = vector.transform(x_test)

In [20]:
clf = MultinomialNB(alpha=0.1).fit(xtrain,y_train)

In [21]:
pre = clf.predict(xtrain)
pre

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  0.,  1.,
        1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  0.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1

In [22]:
clf.predict_proba(xtrain)

array([[1.86725951e-04, 2.46528610e-04, 9.99566745e-01],
       [4.48411633e-07, 5.38194703e-06, 9.99994170e-01],
       [2.15452033e-02, 2.77530738e-02, 9.50701723e-01],
       ...,
       [6.04800428e-06, 4.59655451e-06, 9.99989355e-01],
       [7.24469847e-04, 2.89587356e-04, 9.98985943e-01],
       [4.33254418e-06, 1.21658741e-05, 9.99983502e-01]])

In [23]:
from sklearn import metrics
pre = clf.predict(xtrain)
print('训练数据准确率：',metrics.f1_score(pre, y_train, average='micro'))
pre = clf.predict(xtest)
print('测试数据准确率：',metrics.f1_score(pre, y_test, average='micro'))


训练数据准确率： 0.9723889555822329
测试数据准确率： 0.9243697478991597


In [25]:
xtrain.shape,xtest.shape

((833, 475), (357, 475))